## Практика

In [1]:
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

### Задача 1

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [2]:
# Зачитываем файл с данными в наш датафрейм
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=',')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Напишем функцию, которая на основе двух колонок "Результат анализа" и "Норматив" создает колонку "Выводы" с расшифровкой результата анализа

In [3]:
def translate_results(df):
    for i in range(len(df)):
        if df['Результат анализа'].iloc[i] == 'б/цвета':
            df['Результат анализа'].iloc[i] = '0'
        result = float(df['Результат анализа'].iloc[i])
        normativ = df['Норматив'].iloc[i].replace(',', '.')
        splitlist = normativ.split(' ')
        if splitlist[0] == 'в' and splitlist[1] == 'пределах':
            ranges = splitlist[2].split('-')
            if result >= float(ranges[0]) and result <= float(ranges[1]):
                df['Выводы'] = 'Норма'
            else: 
                df['Выводы'] = 'НЕ НОРМА'
        elif splitlist[0] == 'не' and splitlist[1] == 'более':
            if '-' in splitlist[2]:
                ranges = splitlist[2].split('-')
                if result <= float(ranges[-1]):
                    df['Выводы'].iloc[i] = 'Норма'
                else: 
                    df['Выводы'].iloc[i] = 'НЕ НОРМА'
            else:
                if result <= float(splitlist[2]):
                    df['Выводы'].iloc[i] = 'Норма'
                else: 
                    df['Выводы'].iloc[i] = 'НЕ НОРМА'
        else:
            df['Выводы'].iloc[i] = 'NA'

In [4]:
# Применим функцию к нашему датафрейму
translate_results(df)

In [5]:
# Назначим колонку "Показатель" как колонку с индексом
df = df.set_index('Показатель')

In [6]:
# Выведем получившийся датафрейм
df

,Единица измерений,Результат анализа,Норматив,Выводы
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,0,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


#### Вывод: Только для двух показателей вода не соответствует норме: "Нефть" и "Окисляемость перманганатная"

## Теория вероятности. События


Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2

В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [7]:
# Напишем функцию, которая возвращает кортеж из 3 взятых последовательно фруктов
# В функции реализуем цикл из трех фруктов извлекаемых из ящика. Важно, что количество отсавшихся будет уменьшаться
def fruits_take():
    taken = []
    box = ['orange']*5 + ['apple']*4
    for i in range(3):
        #print(i, box)
        take_one = random.choice(box)
        taken.append(take_one)
        box.remove(take_one)
    return tuple(taken)

In [8]:
# Задаем количество экспериментов = 1000
n = 1000

# Формируем выборку
sample = pd.DataFrame({"fruits": list(map(lambda item: fruits_take(), range(n)))
})

# Считаем статистику по выборке

P_oranges = len(sample[sample['fruits'] == ('orange', 'orange', 'orange')])/sample.shape[0]
print(P_oranges)

0.114


Увеличим выборку до 10к

In [9]:
# Задаем количество экспериментов = 10000
n = 10000

# Формируем выборку
sample = pd.DataFrame({"fruits": list(map(lambda item: fruits_take(), range(n)))
})

# Считаем статистику по выборке

P_oranges = len(sample[sample['fruits'] == ('orange', 'orange', 'orange')])/sample.shape[0]
print(P_oranges)

0.1177


Увеличим выборку до 1 млн

In [10]:
# Задаем количество экспериментов = 10000
n = 1000000

# Формируем выборку
sample = pd.DataFrame({"fruits": list(map(lambda item: fruits_take(), range(n)))
})

# Считаем статистику по выборке

P_oranges = len(sample[sample['fruits'] == ('orange', 'orange', 'orange')])/sample.shape[0]
print(P_oranges)

0.119181


#### Вывод: При достаточно большом количестве экспериментов мы получаем ответ близкий к теоретическому 0.119

### Задача 3

Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [11]:
# Напишем функцию, которая возвращает реализует проверку деталей
# Будем использовать цикл while, который пополняет список деталями пока не попадется стандартная деталь. Далее переведем список в кортеж
def details():
    taken = []
    box = ['standard']*7 + ['non-standard']*3
    while 'standard' not in taken:
        take_one = random.choice(box)
        taken.append(take_one)
        box.remove(take_one)
    return tuple(taken)

In [12]:
# Задаем количество экспериментов = 1000
n = 1000

# Формируем выборку
sample = pd.DataFrame({"details": list(map(lambda item: details(), range(n)))
})

# Считаем статистику по выборке

P_two = len(sample[sample['details'] == ('non-standard', 'standard')])/sample.shape[0]
print(P_two)

0.227


Увеличим выборку до 10к

In [13]:
# Задаем количество экспериментов = 10000
n = 10000

# Формируем выборку
sample = pd.DataFrame({"details": list(map(lambda item: details(), range(n)))
})

# Считаем статистику по выборке

P_two = len(sample[sample['details'] == ('non-standard', 'standard')])/sample.shape[0]
print(P_two)

0.2303


Увеличим выборку до 1 млн

In [14]:
# Задаем количество экспериментов = 100000
n = 1000000

# Формируем выборку
sample = pd.DataFrame({"details": list(map(lambda item: details(), range(n)))
})

# Считаем статистику по выборке

P_two = len(sample[sample['details'] == ('non-standard', 'standard')])/sample.shape[0]
print(P_two)

0.232916


#### Вывод: При достаточно большом количестве экспериментов мы получаем ответ близкий к теоретическому 0.23333